In [8]:
import numpy as np
from QuantumEngine import *

In [13]:
MoveCode = dict({0: "FAIL", 
                 1: "SUCCESS", 
                 2: "X WIN", 
                 3: "O WIN", 
                 4: "TIE"
                })

class QT3:
    def __init__(self):
        self.reset()
        self.winning_combinations = [[0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6]]

    def reset(self, seed=0):
        self.occupied = np.zeros(9)
        self.board = np.zeros(9)
        self.current_player = 'X'
        self.move_history = []
        
        # Each quantum 'tuple' (e.g. a pair in superposition, or three sites in an entangled state)
        # get their own separate Hilbert Space.
        self.HilbertSpaces = []
        
        # Keep track of which Hilbert Space a square is assigned to
        self.squares_to_H = {}
        # Keep track of which squares belong to a Hilbert Space
        self.H_to_squares = {}
        np.random.seed(seed)
        
    def isEmpty(self, square):
        if( self.occupied[square] == 0 ):
            return True
        return False

    def isBoardFull(self):
        for s in range(9):
            if( self.isEmpty(s) ):
                return False
        return True
    
    def isClassical(self):
        return (len(self.HilbertSpaces) == 0)

    def hasWinner(self):
        
        winners = []
        board = [self.get_symbol_for_distribution(self.get_distribution(i)) for i in range(9)]
        for c in self.winning_combinations:
            if( board[c[0]] == board[c[1]] and board[c[1]] == board[c[2]] and (board[c[0]] == 'X' or board[c[0]] == 'O') ):
                winners.append( 0 if board[c[0]] == 'X' else 1 )

        if( len(winners) == 0 ):
            # No winnner, but board is full -> Draw
            if( self.isBoardFull() ):
                return 0

            # No winner and board not full -> Game not done yet
            return -1

        mean_winner = int(np.mean(winners))
        if( mean_winner == 0.5 ):
            return 0
        elif( mean_winner < 0.5 ):
            return 1
        else:
            return 2

    def project(self):
        # Nothing to do
        if( self.isClassical() ):
            return

        # For each Hilbert Space, do the projection
        for H,h in enumerate(self.HilbertSpaces):
            squares = self.H_to_squares[h]
            
            all_states = [s for s in H.state]
            all_probabilities = [np.abs(H.state[s])**2 for s in all_states]
            
            # Pick according to the all_probabilities
            projected_state = np.random.choice(range(len(all_probabilities)), p=all_probabilities)
        
 
        self.state = {all_states[projected_state] : 1}
        self.probabilities = [0 if self.isEmpty(s) else 1 for s in range(9)]
        
    def is_legal_move(self, move):
        if( len(move) == 1 ):
            if( not self.isEmpty(move[0]) ):
                return False
            return True

        if( not self.isEmpty(move[0]) or not self.isEmpty(move[1])):
            return False

        if( move[0] == move[1] ):
            return False

        return True

    def get_legal_moves(self):
        moves = []
        # Classical moves
        for i in range(9):
            if( self.is_legal_move([i])):
                moves.append([i])

        if( self.quantum ):
            # Quantum moves
            for i in range(9):
                for j in range(i+1,9):
                    if( self.is_legal_move([i,j]) ):
                        moves.append([i,j])

        return moves

    def get_distribution(self, square):
        distribution = []
        for token in ['.', 'X', 'O']:
            all_states = [s for s in self.state if s[square] == token]
            weight = np.sum([np.abs(self.state[s])**2 for s in all_states])
            distribution.append(weight)
        return distribution

    def get_symbol_for_distribution(self, distribution):
        if( distribution[0] > 0.99 ):
            return '.'
        if( distribution[1] > 0.99 ):
            return 'X'
        if( distribution[2] > 0.99 ):
            return 'O'

        if( distribution[0] > 0 and distribution[1] > 0 and distribution[2] == 0 ):
            return 'x'
        if( distribution[0] > 0 and distribution[2] > 0 and distribution[1] == 0 ):
            return 'o'

    def print_board(self):
        for i in range(3):
            line = "\t "
            for j in range(3):
                index = 3*i + j
                token = self.get_symbol_for_distribution( self.get_distribution(index) )
                line += token
                if( j != 2 ):
                    line += "  |  "
            print(line)

    def print_probabilities(self):
        for i in range(3):
            line = "\t %.1f | %.1f | %.1f"%(self.probabilities[3*i + 0], self.probabilities[3*i +1], self.probabilities[3*i +2])
            print(line)
        
    def doClassicalMove(self, site, s):
        if( self.board[site] != 0 ):
            return False
        
        self.board[site] = s
        self.occupied[site] = 1
        return True

    def doQuantumMove(self, site1, site2, s):
        
        # If neither square has an assigned Hilbert space
        if( self.square_to_H.get(site1,-1) == -1 and self.square_to_H.get(site2,-1) == -1 ):
            # Make a new Hilbert space for these qutrits
            newH = HilbertSpace([3,3])
            
            # Add to list of Hilbert spaces
            self.HilbertSpaces.append(newH)
            
            # Assign each of these squares to point to that Hilbert space
            self.square_to_H[site1] = len(self.HilbertSpaces)-1
            self.square_to_H[site2] = len(self.HilbertSpaces)-1
            
            # If both squares were empty
            # TODO: Do this through measurement of state
            if( self.occupied[site1] == 0 and self.occupied[site2] == 0 ):
                # Make a superposition
                newH.setProductState([s,0])
                sqrtswap = newH.constructSqrtSwap(0,1)
                newH.applyUnitary(sqrtswap)
                
            elif( (self.occupied[site1] == 0 and self.occupied[site2] != 0) or
                 (self.occupied[site2] == 0 and self.occupied[site1] != 0) ):
                
                # Check which site was the occupied one
                theOccupiedSite = site1 if self.occupied[site1] != 0 else site2
                # Determine opponent's symbol
                o = 1 if s == 2 else 2
                
                newH.setProductState([o,s] if theOccupiedSite == site1 else [s,o])
                sqrtswap = newH.constructSqrtSwap(0,1)
                newH.applyUnitary(sqrtswap)
                
            return True
            
        # If one of the selected squares is in a quantum pair
        elif( (self.square_to_H.get(site1,-1) != -1 and self.square_to_H.get(site2,-1) == -1) or
            (self.square_to_H.get(site2,-1) != -1 and self.square_to_H.get(site1,-1) == -1) ):

            # Find out which one was in a pair
            inPair = site1 if self.square_to_H.get(site1,-1) != -1 else site2
            # Get the Hilbert space
            H = self.HilbertSpaces[ self.square_to_H[inPair] ]
            # Extend it by another qutrit
            H.extend([3])
            # The first qutrit in this Hilbert space is now the new site
            # Find out which index the other square has; for now just use the second
            sqrtswap = H.constructSqrtSwap(0,1)
            H.applyUnitary(sqrtswap)
            return True
            
        return False
            
    def doMove(self, move):
        
        if len(move) == 1:
            site = int(move[0])
            move_success = self.doClassicalMove(site, 1 if self.current_player == 'X' else 2)
            
        if len(move) == 2:
            site1 = int(move[0])
            site2 = int(move[1])
            move_success = self.doQuantumMove(site1, site2, 1 if self.current_player == 'X' else 2)
            
        if( move_success ):
            # Add move to history
            self.move_history.append(move)

            # Switch player
            self.current_player = 'X' if self.current_player == 'O' else 'O'

            # Check if full
            if( self.isBoardFull() ):
                self.project()

            # Check for winner
            self.winner = self.hasWinner()
            if( self.winner != -1 ):
                self.over = True
                
    def undo_move(self):
        move_history = self.move_history
        self.reset()
        for move in move_history[:-1]:
            self.do_move(move)

### Demonstrate a simple game

In [12]:
newGame = QT3()

# 1 = X, 2 = O, 0 = E
newGame.doQuantumMove(0,8,1)
print("First Hilbert space")
print(newGame.HilbertSpaces[0].state)

# Put an O on site 3
newGame.doClassicalMove(3,2)
# Create entangled pair
newGame.doQuantumMove(4,3,1)

print("Second Hilbert space")
print(newGame.HilbertSpaces[1].state)

# And finally add another superposition for O
newGame.doQuantumMove(6,7,2)
print("Third Hilbert space")
print(newGame.HilbertSpaces[2].state)

# Now create a 3-site state
newGame.doQuantumMove(5,6,1)
print("Third Hilbert space -- updated")
print(newGame.HilbertSpaces[2].state)

print("Norm")
print(newGame.HilbertSpaces[2].state.conj().T.dot(newGame.HilbertSpaces[2].state))

First Hilbert space
  (1, 0)	(0.5-0.5j)
  (3, 0)	(0.5+0.5j)
Second Hilbert space
  (5, 0)	(0.5+0.5j)
  (7, 0)	(0.5-0.5j)
Third Hilbert space
  (2, 0)	(0.5-0.5j)
  (6, 0)	(0.5+0.5j)
Third Hilbert space -- updated
  (2, 0)	(0.5-0.5j)
  (6, 0)	0.5j
  (18, 0)	(0.5+0j)
Norm
  (0, 0)	(1+0j)
